In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
pip install kaggle -q

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rm1000/lung-cancer-histopathological-images

In [ ]:
import zipfile

with zipfile.ZipFile("lung-cancer-histopathological-images.zip", "r") as zip_ref:
    zip_ref.extractall("lung_cancer_dataset")

In [ ]:
!ls lung_cancer_dataset

adenocarcinoma	benign	squamous_cell_carcinoma


In [ ]:
# Importações
import os
import random

import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
root = '/content/lung_cancer_dataset'
classes = os.listdir(root)

for i in range(3):
    print(f"# of images in class {classes[i]}: {len(os.listdir(root+'/'+classes[i]))}")

# of images in class squamous_cell_carcinoma: 5000
# of images in class adenocarcinoma: 5000
# of images in class benign: 5000


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Redimensionamento das imagens
    transforms.ToTensor(), # Transformação das imagens para tensores
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Normalização das imagens
])

In [ ]:
dataset = datasets.ImageFolder(root=root, transform=transform)

train_size, val_size = int(0.8 * len(dataset)), int(0.2 * len(dataset)) # 80% das imagens vai para treino, enquanto que 20% para teste

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # shuffle para evitar memorização (overfitting)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
from torchvision import datasets, transforms, models

# Definindo o dispositivo (GPU ou CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Carregar o modelo ResNet-50 pré-treinado
model_resnet = models.resnet50(pretrained=True)

# Substituindo a última camada para classificar 3 classes
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_ftrs, 3)

# Movendo o modelo para o dispositivo
model_resnet.to(device)

In [ ]:
# Definindo o critério de perda (CrossEntropyLoss) e o otimizador (SGD)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9)

print(model_resnet)  # Para verificar a arquitetura modificada

In [ ]:
dataset = datasets.ImageFolder(root=root, transform=transform)

train_size, val_size = int(0.8 * len(dataset)), int(0.2 * len(dataset)) # 80% das imagens vai para treino, enquanto que 20% para teste

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # shuffle para evitar memorização (overfitting)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
for epoch in range(50):
    model_resnet.train()  # Colocar o modelo em modo de treinamento
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Transferir para o dispositivo

        optimizer.zero_grad()  # Zerar os gradientes

        outputs = model_resnet(inputs)  # Passar as entradas pelo modelo
        loss = criterion(outputs, labels)  # Calcular a perda
        loss.backward()  # Retropropagar a perda
        optimizer.step()  # Atualizar os pesos

        running_loss += loss.item()  # Acumular a perda

    # Exibir a perda média por época
    print(f"Epoch: {epoch+1} | Loss: {running_loss / len(train_loader)}")

    # Salvar o modelo treinado após cada época
    torch.save(model_resnet.state_dict(), f"resnet50_epoch{epoch+1}.pth")

Epoch: 1 | Loss: 0.13256405617545047
Epoch: 2 | Loss: 0.03211761977244169
Epoch: 3 | Loss: 0.012328627890984838
Epoch: 4 | Loss: 0.009102295946717883
Epoch: 5 | Loss: 0.008005544987934021
Epoch: 6 | Loss: 0.0037680816483334637
Epoch: 7 | Loss: 0.0037519598958121302
Epoch: 8 | Loss: 0.002848215024142216
Epoch: 9 | Loss: 0.0020174801901642544
Epoch: 10 | Loss: 0.003176094412289482
Epoch: 11 | Loss: 0.0013405501421754403
Epoch: 12 | Loss: 0.001817594485245839
Epoch: 13 | Loss: 0.0011172886414278764
Epoch: 14 | Loss: 0.0009844741620569646
Epoch: 15 | Loss: 0.001070207129935928
Epoch: 16 | Loss: 0.0009020925004491195
Epoch: 17 | Loss: 0.0014589463844992376
Epoch: 18 | Loss: 0.0009329724078124855
Epoch: 19 | Loss: 0.0005065910139674087
Epoch: 20 | Loss: 0.0006328377929045625
Epoch: 21 | Loss: 0.000929600194428834
Epoch: 22 | Loss: 0.000488511000234818
Epoch: 23 | Loss: 0.0013256147572113454
Epoch: 24 | Loss: 0.0006082726490640197
Epoch: 25 | Loss: 0.000438292340610739
Epoch: 26 | Loss: 0.000

In [ ]:
import torch
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Colocar o modelo em modo de avaliação
model_resnet.eval()

correct = 0
total = 0

# Listas para armazenar as predições e os rótulos reais
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Mover para o dispositivo (GPU/CPU)

        # Passar as imagens pelo modelo
        outputs = model_resnet(inputs)

        # Obter as predições
        _, predictions = torch.max(outputs, 1)

        # Armazenar as predições e os rótulos reais
        all_preds.extend(predictions.cpu().numpy())  # Transferir para CPU e converter para numpy
        all_labels.extend(labels.cpu().numpy())  # Transferir para CPU e converter para numpy

        # Calcular a acurácia
        correct += (predictions == labels).sum().item()
        total += len(labels)

# Calcular a acurácia
accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.4f}")

# Gerar a matriz de confusão
conf_mat = confusion_matrix(all_labels, all_preds)

# Visualizar a matriz de confusão
class_names = ['adenocarcinoma', 'benign', 'squamous_cell_carcinoma']  # Atualize com seus nomes de classe

plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Classe Predita')
plt.ylabel('Classe Real')
plt.title('Matriz de Confusão')
plt.show()

In [ ]:
from PIL import Image

# Definir as transformações para as imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Certifique-se de que a imagem tem o tamanho correto
    transforms.ToTensor(),  # Converte a imagem para tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normaliza com os valores típicos para RGB
])

# Carregar suas três imagens
image_paths = ['lungaca1.jpeg', 'lungn1.jpeg', 'lungscc1.jpeg']  # Altere para o caminho das suas imagens
images = []

for image_path in image_paths:
    image = Image.open(image_path).convert("RGB")  # Abra e converta para RGB
    images.append(transform(image))  # Aplique a transformação e adicione à lista

# Empacotar as imagens em um tensor (batch)
images_tensor = torch.stack(images)  # Agora, temos um tensor com shape (3, 3, 128, 128)

In [ ]:
# Certifica que o modelo está na GPU, se disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_resnet.to(device)

# Coloca o modelo em modo de avaliação
model_resnet.eval()

# Move o tensor de imagens para o mesmo dispositivo do modelo
images_tensor = images_tensor.to(device)

# Realizar a predição sem calcular os gradientes
with torch.no_grad():
    outputs = model_resnet(images_tensor)
    probabilities = torch.softmax(outputs, dim=1)
    predicted_classes = torch.argmax(probabilities, dim=1)

    class_names = ['adenocarcinoma', 'benign', 'squamous_cell_carcinoma']

    for i, predicted_class in enumerate(predicted_classes):
        print(f"Imagem {i+1}: Classe prevista: {class_names[predicted_class.item()]}")
        print(f"Probabilidades: {probabilities[i]}")

Imagem 1: Classe prevista: adenocarcinoma
Probabilidades: tensor([1.0000e+00, 3.1408e-06, 7.8519e-07], device='cuda:0')
Imagem 2: Classe prevista: benign
Probabilidades: tensor([8.7386e-08, 1.0000e+00, 4.4555e-11], device='cuda:0')
Imagem 3: Classe prevista: squamous_cell_carcinoma
Probabilidades: tensor([7.1285e-05, 7.2613e-06, 9.9992e-01], device='cuda:0')
